<a href="https://colab.research.google.com/github/woodsj1206/Train-Yolov8-Image-Classification-On-Custom-Dataset/blob/main/Train_Yolov8_Image_Classification_On_Custom_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Step 1: Checking GPU Availability
Check if a GPU is available in your Google Colab environment. A GPU can significantly accelerate the training process of deep learning models like YOLOv8.

In [ ]:
!nvidia-smi

#Step 2: Unzipping Dataset Files
Unzip the dataset file containing images. These files are typically compressed to save space and are now being extracted for use in training the YOLOv8 model.

In [ ]:
# Unzip the zip file containing the images
!unzip -q 'REPLACE_WITH_YOUR_IMAGE_ZIP_FILE_PATH' -d '/content/images'

#Step 3: Installing Ultralytics
Install Ultralytics to work with YOLO models.

In [ ]:
# Install Ultralytics library
!pip install ultralytics

# Import necessary modules from Ultralytics
from ultralytics import YOLO

#Step 4: Mounting Google Drive
Mount Google Drive to the Colab environment, enabling access to files stored in your Google Drive.

In [ ]:
# Import the drive module from google.colab
from google.colab import drive

# Mount Google Drive to '/content/Google_Drive'
drive.mount('/content/Google_Drive')

#Step 5: Define Root Directory and Create Subdirectories
Define the root directory for your project and create necessary subdirectories to organize your data.

In [ ]:
import os

# Prompt user for the path
PATH = input("Enter the desired path: ")

ROOT_DIR = f'/content/Google_Drive/MyDrive/{PATH}'

# Create directory for data
DATA_DIR = os.path.join(ROOT_DIR, 'data')

# Create directory for trainig and validation
DATA_VAL_DIR = os.path.join(DATA_DIR, 'val')
DATA_TRAIN_DIR = os.path.join(DATA_DIR, 'train')

# Create directory for testing
TESTING_DIR = os.path.join(ROOT_DIR, 'testing')

# Create the directories if the root directory doesn't exist
if not os.path.exists(ROOT_DIR):
    os.makedirs(ROOT_DIR)

    os.makedirs(DATA_DIR)
    os.makedirs(DATA_VAL_DIR)
    os.makedirs(DATA_TRAIN_DIR)

    os.makedirs(TESTING_DIR)

    print(f"Root directory '{ROOT_DIR}' created successfully.")
else:
    print(f"Root directory '{ROOT_DIR}' already exists.")

#Step 6: Organize Dataset for Training and Validation
Organize the dataset by moving images and splitting them into separate directories for training and validation.

In [ ]:
import shutil

# Replace placeholder with the actual path to the folder containing subfolders of images
PARENT_FOLDER_PATH = 'REPLACE_WITH_YOUR_FOLDER_PATH'

# Get list of image folders
image_folders = os.listdir(PARENT_FOLDER_PATH)

for folder in image_folders:
    # Create directories for validation and training data
    val_dir = os.path.join(DATA_VAL_DIR, folder)  # Validation directory path
    train_dir = os.path.join(DATA_TRAIN_DIR, folder)  # Training directory path

    os.makedirs(val_dir)  # Create validation directory
    os.makedirs(train_dir)  # Create training directory

    images_path = os.path.join(PARENT_FOLDER_PATH, folder)

    # Get list of image files contained in a folder
    image_files = os.listdir(images_path)

    # Determine the number of files for the training set
    train_count = int(len(image_files) * 0.7)  # 70% of data used for training, 30% used for validation

    # Move files to the training directory
    for file in image_files[:train_count]:
        shutil.move(os.path.join(images_path, file), os.path.join(train_dir, file))

    # Move files to the validation directory
    for file in image_files[train_count:]:
        shutil.move(os.path.join(images_path, file), os.path.join(val_dir, file))


#Step 7: Train the YOLOv8 Model
Train the YOLOv8 model using the provided dataset configuration and save the training results.

In [ ]:
# Load a pre-trained YOLOv8 model
model = YOLO('yolov8n-cls.pt')

# Train the model using the provided dataset configuration
model_results = model.train(data=DATA_DIR, epochs=20)

# Save the training results
shutil.make_archive(base_dir='/content/runs', root_dir='/content/runs', format='zip', base_name=f'{ROOT_DIR}/runs')

#Step 8: View Training Results
Display the training results and the confusion matrix generated during the training process.

In [ ]:
from IPython.display import Image

# Display the training results
Image('runs/classify/train/results.png')

In [ ]:
# Display the confusion matrix
Image('runs/classify/train/confusion_matrix.png')

#Step 9: Process Image Files
Process image files using the trained YOLOv8 model and save them in the Colab environment.

**NOTE:** Ensure you have uploaded your image files to the testing folder before proceeding with this step.

In [ ]:
import cv2
import numpy as np

# Define the directory where you want to save the images
OUTPUT_DIR = '/content/output'

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Path to the trained YOLO model file
model_file = "/content/runs/classify/train/weights/best.pt"

# Load the YOLO model
model = YOLO(model_file)

# List all files in the testing directory
files = os.listdir(TESTING_DIR)

for image_file in files:
    # Construct the full path to the image file
    image_path = os.path.join(TESTING_DIR, image_file)

    # Load image using OpenCV
    image = cv2.imread(image_path)

    # Get the height and width of the image
    height, width, _ = image.shape

    # Perform inference using the model
    results = model.predict(image)

    # Get class names from the model's output
    names = results[0].names

    # Get probabilities of each class
    probabilities = results[0].probs.data.tolist()

    # Get the index of the class with the highest probability
    top_prediction_index = np.argmax(probabilities)

    # Format label with class name and confidence score
    label = f'{names[top_prediction_index]}: {probabilities[top_prediction_index]:.2f}'

    # Scale font size and location based on image size
    font_scale = min(height, width) / 1000
    thickness = max(1, int(min(height, width) / 1000))

    # Define text size and baseline to calculate the rectangle size
    (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)
    text_baseline = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)[1]

    location = (0, text_height + text_baseline)

    # Define rectangle coordinates
    rectangle_start = (location[0], location[1] - text_height - text_baseline)
    rectangle_end = (location[0] + text_width, location[1] + text_baseline)

    # Draw filled rectangle behind the text
    cv2.rectangle(image, rectangle_start, rectangle_end, (255, 0, 0), cv2.FILLED)

    # Put text on the image with the detected label
    cv2.putText(image, label, location, cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 255, 255), thickness)

    # Display the image with the detected label
    output_file = os.path.join(OUTPUT_DIR, image_file)  # Output file path
    cv2.imwrite(output_file, image)  # Save the modified image

#Step 10: Display Processed Images
Use IPython.display to display the processed images within the Colab environment.


In [ ]:
from IPython.display import display

# List all files in the output directory
output_images = os.listdir(OUTPUT_DIR)

for image in output_images:
    # Construct the full path to the image file
    image_path = os.path.join(OUTPUT_DIR, image)

    # Display the image using IPython.display
    display(Image(image_path))